In [7]:
import os
import pandas as pd
from tqdm import tqdm

data = "updatedCveDB.csv"

if not os.path.exists(data):
    print(f"[INFO] Downloading data from MITRE...")
    url = "https://cve.mitre.org/data/downloads/allitems.csv"
    df = pd.read_csv(url, skiprows=2, encoding='latin-1', dtype=str)
    total_rows = len(df)
    with tqdm(total=total_rows, desc="Processing data") as pbar:
        df.to_csv(data, index=False)
        pbar.update(total_rows)

# Select only the first three columns
df = df.iloc[:, :3]

# Process your data
# Remove rows where description starts with disclaimers, remove rows with missing description
df = df.dropna(subset=['Description'])
df = df[~df['Description'].str.startswith('**')]

# Save the processed data
df.to_csv('updatedCveDB.csv', index=False)

print(f"[INFO] Data saved to updatedCveDB.csv")

[INFO] Downloading data from MITRE...


KeyboardInterrupt: 

In [5]:
df.head()

,Name,Status,Description
7,CVE-1999-0001,Candidate,ip_input.c in BSD-derived TCP/IP implementatio...
8,CVE-1999-0002,Entry,Buffer overflow in NFS mountd gives root acces...
9,CVE-1999-0003,Entry,Execute commands as root via buffer overflow i...
10,CVE-1999-0004,Candidate,"MIME buffer overflow in email clients, e.g. So..."
11,CVE-1999-0005,Entry,Arbitrary command execution via IMAP buffer ov...
